## H(curl) - AMG

We present the amg from [Reitzinger and Schöberl: An algebraic multigrid method for finite element discretizations with edge elements](https://onlinelibrary.wiley.com/doi/abs/10.1002/nla.271?casa_token=SGxs8UGF--IAAAAA:53O8vbFJpEkXyuSu4T2yzP7BKBJecdNoFdEvLqUKT_ZRUMn0U5FM--SqGXRiQu38et4xuMPg6cPUgfUBoQ).

It is based on a surrogate matrix for a weighted $H(\operatorname{curl})$ norm discretized by lowest order Nedelec elements:

$$
\| u \|_{L_2, \sigma}^2 + \| \operatorname{curl} u \|_{L_2, \nu}^2
\approx \sum_E w_E \, \Big(\int_E u_{\tau} \Big)^2 + 
\sum_F w_F \, \Big(\int_F \operatorname{curl}_n u \Big)^2
$$

The edge-weights stem from the $L_{2,\sigma}$ norm. One could take
the diagonal of the edge-element mass matrix, or also Schur complements with respect to the edges. The $\operatorname{curl}$-semi-norm is represented by the face terms. The weights can be computed by lumping the lowest-order Raviart-Thomas mass matrix. 

The smoother is a Hiptmair smoother, where a Gauss-Seidel smoother is combined with another Gauss-Seidel smoother for the potential space.

The key is a coarsening which preserves the de Rham sequence over all levels, such that Hiptmair's smoother is effective also on coarser levels.


$$
\!\!\!\!\!\!\!\!\!
\begin{array}{ccccccc}
 & B_{\operatorname{grad}} & 
 & B_{\operatorname{curl}} &  
 & B_{\operatorname{div}} & \\[-0.5em]
V^v & 
-\!\!\!\longrightarrow 
& V^e & 
-\!\!\!-\!\!\!\longrightarrow 
& V^f & 
-\!\!\!-\!\!\!\longrightarrow 
& V^c \\[1em]
\;\;\;\;\downarrow \Pi^W & &
\;\;\;\;\downarrow \Pi^V & & 
\;\;\;\;\downarrow \Pi^Q & & 
\;\;\;\;\downarrow \Pi^S \\[1em]
 & B_{\operatorname{grad}}  & 
 & B_{\operatorname{curl}} &  
 & B_{\operatorname{div}} & \\[-0.5em]
V^v_{coarse} & 
-\!\!\!\longrightarrow 
& V^e_{coarse} & 
-\!\!\!-\!\!\!\longrightarrow 
& V^f_{coarse} &
-\!\!\!-\!\!\!\longrightarrow 
& V^c_{coarse}
\end{array}
$$



The coarsening of edges is derived from coarsening of vertices. $E_{IJ}$ is a coarse grid edge if and only if $I \neq J$, and there are fine grid vertices $i$ and $j$ s.t.:

$$
I = Ind(i), \quad J = Ind(j), \qquad E_{ij} \mbox{ is a fine grid edge}
$$


<img src="agglomerates-hc.png" alt="Alternative text" width="300" align="center"/>





More recent, robust coarsening strategies are developed in [B. Schwarzenbacher: Robust algebraic solvers for electromagnetics, Master's Thesis](https://repositum.tuwien.at/handle/20.500.12708/1351)


## H(curl) - AMG in NGSolve
This amg method is implemented as `hcurlamg` preconditioner in NGSolve.

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw

coil = Cylinder( Axes( (0,0,-0.4), Z, X), h=0.8,r=0.4) \
    - Cylinder( Axes( (0,0,-0.4), Z, X), h=0.8,r=0.2)
box = Box( (-2,-2,-2), (2,2,2) )

coil.faces.col=(1,0,0)
coil.faces.maxh=0.1
coil.solids.name="coil"
box.faces.col=(0,0,1,0.3)
box.faces.name="outer"
air = box - coil
shape = Glue( [coil,air] )
Draw (shape);

In [ ]:
mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=0.2)) # .Curve(3)
Draw (mesh);

In [ ]:
fes = HCurl(mesh, order=2, nograds=True) # , dirichlet="outer")
print ("ndof=", fes.ndof)
u,v = fes.TnT()

with TaskManager():
    mu = 4*pi*1e-7
    a = BilinearForm(1/mu*curl(u)*curl(v)*dx + 1e-6/mu*u*v*dx \
                     + 1e8*u.Trace()*v.Trace()*ds("outer"))
    # pre = preconditioners.HCurlAMG(a)
    pre = preconditioners.MultiGrid(a, coarsetype="hcurlamg", coarseflags={ "steps" : 5 }) 
    a.Assemble()
    f = LinearForm( CF((y,-x,0))*v*dx("coil", bonus_intorder=5)).Assemble()

In [ ]:
gfu = GridFunction(fes)
from ngsolve.krylovspace import CGSolver

with TaskManager():
    inv = CGSolver(a.mat, pre.mat, plotrates=True, maxiter=100)
    gfu.vec[:] = inv*f.vec

In [ ]:
ea = { "euler_angles" : (-60, 0, -11) }
clipping = { "clipping" : { "y":1, "z":0, "dist":0.5} }

s = 0.1*2
N = 10
p = [(-s+2*s*i/N,-s+2*s*j/N,-s+2*s*k/N) for i in range(1,N) for j in range(1,N) for k in range(1,N)]

fieldlines = curl(gfu)._BuildFieldLines(mesh, p, num_fieldlines=N**3//5, randomized=True, length=2)

Draw(curl(gfu), mesh,  "X", draw_vol=False, draw_surf=True, objects=[fieldlines], \
     min=0, max=1e-8, autoscale=False, settings={"Objects": {"Surface": False}},
    **ea, **clipping);

In [ ]:
help (preconditioners.MultiGrid)